Misconception Data
===

In [1]:
import json
import os
import re
from collections import Counter
from pathlib import Path

import dotenv
import matplotlib.pyplot as plt
import numpy as np
import openai
import pandas as pd
import requests
import tiktoken

from streamlit_app import data_utils

In [2]:
dotenv.load_dotenv("../.env")
openai.api_key = os.environ["OPENAI_API_KEY"]

In [28]:
data_dir = Path("../data")
assert data_dir.exists()

In [3]:
result = requests.get("https://raw.githubusercontent.com/creature-ai/math-misconceptions/main/misconceptions.json")

In [4]:
misconception_list = json.loads(result.text)
len(misconception_list)

58

In [5]:
df = pd.DataFrame(misconception_list)
df.columns

Index(['Misconception', 'ID', 'Topic', 'Example1',
       'Example1 Graph Description', 'Paper 1', 'Example2',
       'Example2 Graph Description', 'Paper 2', 'Example3',
       'Example3 Graph Description', 'Paper 3', 'Example4',
       'Example4 Graph Description', 'Paper 4'],
      dtype='object')

In [6]:
df

,Misconception,ID,Topic,Example1,Example1 Graph Description,Paper 1,Example2,Example2 Graph Description,Paper 2,Example3,Example3 Graph Description,Paper 3,Example4,Example4 Graph Description,Paper 4
0,when students write incorrectly transpose vari...,MaE46,"Variables, expressions, and operations",Q:\nWrite an algebraic expression to represent...,,"Capraro, M. M., & Joffrion, H. (2006)\np. 7",Q: \nWrite the expression that represents: “Ta...,,"Capraro, M. M., & Joffrion, H. 2006\np.18",Q: \nAt a meeting there are five more women th...,,"Stacey, K & MacGregor M (2014)\n\np. 5","Q: ""z is equal to the sum of 3 and y."" Write t...",,"Stacey, K & MacGregor M (2014)\n\np. 5"
1,when students incorrectly view variables as la...,MaE47,"Variables, expressions, and operations",Q:\nThe following question is about this expre...,,Asquith et al. 2007p. 11,Q:\nDavid is 10 cm taller than Con. Con is h c...,,"Stacey and MacGregor, 1997b\np. 2",Q:\nGiven the equation: a = 28 + b\nWhich of t...,,"Stacey and MacGregor, 1997b\np. 2",Q: \nSue weighs 1 kg less than Chris. Chris ...,,MacGregor and Stacey 1997\np. 6
2,when students believe that two variables in th...,MaE48,"Variables, expressions, and operations","Q: \nh + m + n = h + p + n always, sometimes,...",,Stephens (2005)\np. 1,Q:\nRicardo has 8 pet mice. He keeps them in t...,,Stephens (2005)\np. 2-4,Q:\nAre these number sentences true or false?\...,,Stephens (2005)\np. 1,0,,
3,when students fail to understand that variable...,MaE49,"Variables, expressions, and operations",Q:\nA store is having a sale. They have advert...,,"Bair, S. L., & Rich, B. S. 2011\np. 12",Q:\nWhat can you say about r if r = s + t and ...,,Kuchemann (1978)\np.3,Q: \nWhich is larger task: Can you tell which...,,Kuchemann 1978\np. 5,Q:\nWhat can you say about c if c + d = 10 and...,,Kuchemann (1978)\np.3
4,when students believe that the commutative and...,MaE50,"Variables, expressions, and operations",Q:\nAre these statements true or false? \n2 – ...,,Baroudi 2006\np. 3,Q:\nSimplify: 3(6x - 4) + 2(3x - 3)\nShow your...,,"Bush, 2011\np. 215-216","Q: \nWithout using the calculator, just by loo...",,"Linchevski, L., & Livneh, D. 1999\np. 12",Q: \nBlanca's aquarium has three types of fish...,,"Bush, 2001\np. 217-219"
5,when students hold a rigid understanding of pr...,MaE23,Ratios and proportional reasoning,"Q:\nAt ARCO, gas sells for $1.13 per gallon. T...",,"Kilpatrick, J., Swafford, J. O., & Findell, B....",Q: \nAdrian has conquered only 6 giants in his...,,"Brown, G., & Quinn, R. J. (2006) \np. 5","Q:\n Which of the following drinks, A or B, ...",,Lamon 2012\n(no link),Q: \nConsider two marigolds that were 8 inches...,,"Kilpatrick, J., Swafford, J. O., & Findell, B...."
6,when students show an inability to recognize t...,MaE24,Ratios and proportional reasoning,"Q:\nTo bake donuts, Mariah needs 8 cups of flo...",,"Singh, 2000\np. 17","Q:\nTo bake 6 cakes, you need 15 eggs. Using t...",,"Singh, 2000\np. 19","Q:\nIf 4 cents can buy 6 sweets, how many can ...",,"Singh, 2000\np. 8","Q: \nIn baking a cake, for every 8 eggs, one...",,"Sigh, 2000\np. 18"
7,when students have confusions with ratios writ...,MaE25,Ratios and proportional reasoning,"Q:\nExpress ""2/3 dollars per balloon"" with equ...",,"Kilpatrick, J., Swafford, J. O., & Findell, B....",Q:\n3:2 = 24:x\nFind the value of x\n\nA:\n32 ...,,"Kilpatrick, J., Swafford, J. O., & Findell, B....",Q:\n If 5/8 = x/24 then find x\n\nA:\nx = 5...,,"Brown, G., & Quinn, R. J. (2006) \np. 6",Q:\nGreg conducts a survey of 100 classmates t...,,Bush 2011\np. 188
8,when students present a lack of recognition of...,MaE26,Ratios and proportional reasoning,"Q:\nIf 5 quarters can buy 100 candies, how man...",,Ojose. B. 2015\np. 4,Q: \n(1/3)*a = ?\n\nA:\n1/3a\n[The learner did...,,"Brown, G., & Quinn, R. J. (2006) \np. 6",Q:\nReduce: (3 + 4)/2\n\nA:\n4 ÷ 2 + 3 = 5\n\...,,"Brown, G., & Quinn, R. J. (2006) \np. 7",Q:\nYou had a picture on your computer and you...,,Lamon 2012\n(no link)
9,when

In [27]:
df.to_csv(data_dir / "derived" / "misconceptions.csv")

In [7]:
df.ID.value_counts().head()

ID
         3
MaE47    1
MaE11    1
MaE12    1
MaE13    1
Name: count, dtype: int64

In [8]:
# not sure what's going on with the misconception without an ID
df[~df.ID.str.startswith("MaE")]

,Misconception,ID,Topic,Example1,Example1 Graph Description,Paper 1,Example2,Example2 Graph Description,Paper 2,Example3,Example3 Graph Description,Paper 3,Example4,Example4 Graph Description,Paper 4
16,when students lack understanding and don't see...,-,"Patterns, relationships, and functions",Q: Make a table of values that would produce t...,Q: Make a table of values that would produce t...,"Kalchman, M., & Koedinger, K. R. 2005\np. 3-8","Q:\nFor the function f(x) = 2x^2 + 3x - 1, (i)...","Q:\nFor the function f(x) = 2x^2 + 3x - 1, (i)...","Bair, S. L., & Rich, B. S. 2011\np. 14","Q:\nFor the function f(x) = 2x^2 + 3x - 1, (ii...","Q:\nFor the function f(x) = 2x^2 + 3x - 1, (ii...","Bair, S. L., & Rich, B. S. 2011 \np. 14",Q: \nAfter having solving the equation 2 = 4x ...,,"Kalchman, M., & Koedinger, K. R. (2005)\np. 20"
55,,,,0,,,3,,,,,,12,,
56,,,,1,,,1,,,,,,7,,
57,,,,"Q. Write an expression that represents: ""“Juli...",,,,,,,,,,,


### Prompting tests

In [9]:
tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")

In [10]:
mae_spreadsheet_string = ""
misconception_df = df[df.ID.str.startswith("MaE")]
mae_count = len(misconception_df)
for row in misconception_df.sample(frac=1).itertuples():
    s = f"ID: {row.ID}; Description: {row.Misconception}; Example 1: {row.Example1}; Example 2: {row.Example2}\n"
    # print(s)
    mae_spreadsheet_string += s
n_tokens = len(tokenizer.encode(mae_spreadsheet_string))
n_tokens

11686

In [13]:
lesson_df = data_utils.load_hint_problem_data_from_dir(data_dir / "derived")
lesson_df.sample(n=1)

,unique_index,lesson_code,grade,domain,construct,subconstruct,skill,lesson_index,topic,short_description,...,section,question_index,is_explanation,needs_context,context_string,n_question_tokens,incorrect_answer,description,display_name,lesson_trimmed
5,441,G8.A2.1.1.1,8,A,A2,A2.1,A2.1.1,1,Linear Expressions (1 of 2),Let us learn how to write algebraic expression...,...,2,9,False,False,NaN,37.0,A,Let us learn how to write algebraic expression...,Linear Expressions (grade 8): There are x ball...,An algebraic expression is an expression which...


In [14]:
lesson_df.columns

Index(['unique_index', 'lesson_code', 'grade', 'domain', 'construct',
       'subconstruct', 'skill', 'lesson_index', 'topic', 'short_description',
       'lesson', 'question', 'answer', 'hint', 'has_image', 'section',
       'question_index', 'is_explanation', 'needs_context', 'context_string',
       'n_question_tokens', 'incorrect_answer', 'description', 'display_name',
       'lesson_trimmed'],
      dtype='object')

In [15]:
lesson_df[["unique_index", "lesson_code", "question", "answer", "incorrect_answer"]]

,unique_index,lesson_code,question,answer,incorrect_answer
20,2002,G6.N1.1.2.1,Two hundred fifty three thousand three hundred...,=,2
15,541,G6.S2.1.1.1,"There are 5 candies in a jar, 2 are caramel, 2...",C,B
16,542,G6.S2.1.1.1,"There are 5 candies in a jar, 2 are caramel, 2...",C,A
17,543,G6.S2.1.1.1,"John throws a die. What's more likely, for him...",A,C
18,545,G6.S2.1.1.1,"When throwing a dice, how likely is it to thro...",B,C
19,547,G6.S2.1.1.1,"There are 22 balls, 4 blue, 5 red, 6 yellow an...",C,A
14,476,G7.N5.1.1.1,√225 = __,15,25
13,457,G7.N5.1.1.2,2 x 2 = ___,2^2,4
21,2483,G7.N4.2.1.1,-166 + 37 = ___,-129,129
0,434,G8.A2.1.1.1,A car travels 120 miles in H hours. The expres...,A,B


In [30]:
model = "gpt-3.5-turbo-16k"
prompt = """I’ll give you a spreadsheet with a list of MaEs. 
Each MaE has an ID, a description of the MaE, and 2 examples of the MaE. 
Then I'll show you an incorrect answer to a math question. 
I want you to tell me how many of the {mae_count} MaEs you can identify in the answer, identifying as many as you can.

Spreadsheet:
{mae_spreadsheet_string}

Relevant lesson: {lesson}
Question: {question}
Answer: {answer}
Incorrect Answer: {incorrect_answer}

List MaEs in the incorrect answer along with an explanation for why the MaE is relevant.
"""

results = []
for row in lesson_df.sample(n=5).itertuples():
    formatted_prompt = prompt.format(
        mae_spreadsheet_string=mae_spreadsheet_string,
        mae_count=mae_count,
        lesson=row.lesson_trimmed,
        question=row.question,
        incorrect_answer=row.incorrect_answer,
        answer=row.answer,
    )
    completion = openai.ChatCompletion.create(
        model=model,
        messages=[
            {
                "role": "user",
                "content": formatted_prompt,
            }
        ],
    )
    result = completion["choices"][0]["message"]["content"]
    results.append(result)

In [31]:
results

['- MaE17: The student mistakenly believes that the number of caramel candies and lollipops is equal, rather than recognizing that there are more caramel candies. This is an example of a misconception about proportions and ratios.\n- MaE57: The student did not consider the possibility of the candies being equally likely to be picked, instead assuming that one type of candy had a higher likelihood. This is a failure to understand the concept of probability.\n- MaE26: The student did not accurately represent the different ratios of caramel and lollipops in the jar, which led to their incorrect conclusion. This is a misunderstanding of ratio representation.\n- MaE20: The student did not correctly interpret the given information about the number of candies in the jar, leading to their mistaken answer. This is a failure to accurately understand and represent the situation.\n- MaE5: The student mistakenly compared the decimal values of the candies without considering the number of each type.

In [17]:
m_df = lesson_df.copy()
m_df["gpt_result"] = results

In [18]:
def get_maes(result):
    return re.findall("MaE[0-9]+", result)


mae_lists = []
for result in m_df.gpt_result:
    mae_lists.append(get_maes(result))

In [23]:
m_df["identified_maes"] = [",".join(mae_list) for mae_list in mae_lists]
m_df["n_maes"] = [len(mae_list) for mae_list in mae_lists]

In [20]:
m_df[m_df.n_maes == 0][["question", "answer", "incorrect_answer", "gpt_result", "n_maes"]]

,question,answer,incorrect_answer,gpt_result,n_maes


In [25]:
c = Counter()
for mae_list in mae_lists:
    c.update(mae_list)
c.most_common(5)

[('MaE46', 10), ('MaE57', 4), ('MaE13', 4), ('MaE55', 3), ('MaE18', 3)]

In [26]:
m_df[
    ["lesson_code", "grade", "question", "answer", "incorrect_answer", "gpt_result", "identified_maes", "n_maes"]
].to_csv(data_dir / "derived" / "misconception_identification_example.csv")